In [1]:
!pip install feedparser         # xml 링크에 있는 데이터 전부 가져옴 / RSS에서 xml 태그별 정보 추출(예: title, link, author...)
!pip install newspaper3k        # 인터넷 신문기사 분석(아래 'Article()'을 사용하기 위함)
!pip install konlpy             # 한국어에서 명사만 추출 / 한국어 형태소 분석기(주어진 문장에서 명사만 추출)
import feedparser               # !pip install feedparser
from newspaper import Article   # 신문기사를 타고 들어가서 제목 내용을 가져오게 함 / !pip install newspaper3k
from konlpy.tag import Okt      # 명사, 형용사, 관사 등등 뽑아냄 / !pip install konlpy
from collections import Counter # 몇개의 명사가 있는지 개수를 세는 함수 / 명사 추출 후에 본문에 몇 번 나오는지 확인(TF 계산용)
from operator import eq         # 검색어에 있는 명사가 본문에 있는지 확인 / 동일한 단어인지 여부 판단
from bs4 import BeautifulSoup   # 파싱해왔을 때 본문에 있는 html 태그 제거 / 글에 존재할지 모르는 html 태그 삭제

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=b9341de6d973b2209321910e04777dbe570bf438bb62460aac24e98561793e37
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=4072fde09ad4e6ba9329593433ab4664dc4328248eca76ab11193d1712c634b4
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58d

In [2]:
## [1] 모든 RSS파일(xml형식)을 돌아다니며 기사의 제목과 link를 추출함
# urls는 우리가 검색할 RSS의 목록을 list로 만든
urls=["http://rss.etnews.com/Section901.xml",
      "http://rss.etnews.com/Section902.xml",
      "http://rss.etnews.com/Section903.xml",
      "http://rss.etnews.com/Section904.xml"]

# 아래 함수는 RSS 목록의 list 안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(ulrs):
    array_rss = []      # 함수가 시작하는 시점에 생성되는 빈 리스트. 여기에 모든 기사를 저장
    titles_rss = set()  # '중복 기사 제거' : 기사 제목들의 집합을 구성함. 집합은 중복을 불허하기 때문에 중복 기사가 제거됨

    for url in urls:    # 4개의 RSS 파일을 하나씩 방문(4번 돌것임(901,902,903,904))
        print("[Crawl Rss]", url)         # 현재 어디에 있는지 출력
        parse_rss = feedparser.parse(url) # 현재 url을 파싱해서 그 결과를 parse_rss에 저장
        for p in parse_rss.entries:       # 현재 parse_rss에 있는 모든 entires/기사를 검색
            if p.title not in titles_rss: # '중복기사 제거' : 만약 titles_rss 안에 동일한 제목이 없다면 리스트에 추가
                array_rss.append({'title':p.title, 'link':p.link}) # 기사에서 제목/link 추출 후 리스트에 추가 : 전체 본문을 가져오기 위해 link를 가져옴
                titles_rss.add(p.title)                             # '중복기사 제거' : 집합에 현재 기사 제목이 없을 때만 추
            else:
                print("Duplicated Title:", p.title)
    return array_rss

list_articles = crawl_rss(urls)
print(list_articles)

[Crawl Rss] http://rss.etnews.com/Section901.xml
[Crawl Rss] http://rss.etnews.com/Section902.xml
Duplicated Title: 아이파트너즈, 혁신 스타트업 유럽 진출 지원 시동
[Crawl Rss] http://rss.etnews.com/Section903.xml
Duplicated Title: 아이폰15 시리즈, 품귀현상 나타날 것으로 전망
Duplicated Title: 칼부림부터 학교폭력 대응까지…다재다능한 지능형 CCTV 주목
Duplicated Title: 토요타 HEV 마지막 퍼즐 '신형 프리우스' 연말 韓 상륙
[Crawl Rss] http://rss.etnews.com/Section904.xml
Duplicated Title: 아이파트너즈, 혁신 스타트업 유럽 진출 지원 시동
Duplicated Title: [미래 반도체 유니콘 열전]〈4〉퀄리타스반도체, 대규모 R&D로 '글로벌 인터커넥트 IP 기업' 도전
Duplicated Title: 사공민 GS리테일 점포운영지원실장 “중기·점주 협업, 친환경 편의점 시대 연다”
Duplicated Title: 제주삼다수, 먹는샘물 25년간 부동의 1위 비결은
Duplicated Title: [단독] 수자원공사 해외투자 난관…'원금 회수' 적신호
Duplicated Title: 산업부, '무역·기술 안보 종합전략' 세운다…연내 포럼 구성
Duplicated Title: '100경분의 1초' 빛으로 '전자의 세계' 연 과학자들 노벨 물리학상 수상
Duplicated Title: 정부, 디플정 마중물 '전자정부 클라우드 플랫폼' 구축 착수
Duplicated Title: [에듀플러스]“특성화고, 신산업·신기술 분야에 맞춘 학과 개편 활발…지난 7년간 372개교 902개과 학과 개편”
Duplicated Title: [에듀플러스]특성화고가 진화한다 “기업 협약형 특성화고 설립하고 서울형 마이스터고 전환…취업 품질 높이고 대학 진학도 

In [3]:
## [2] list에 존재하는 모든 링크를 돌아다니며 본문 txt를 추출

# 아래 함수는 하나의 url을 입력받아서, 링크를 타고 들어가, 그 안에 title과 text를 추출한다.
# 디폴트는 한글로 지정한다.
def crawl_article(url, language='ko'):
    print("[Crawl Article]", url)       # 현재 title과 text를 추출한 url을 프린트
    a = Article(url, languate=language) # Article을 사용하여 그 URL을 입력하고, 언어옵션 지정 후 a에 저장
    a.download()   # a에 해당하는 url 기사 다운로드
    a.parse()      # a에 해당하는 url 기사 분석(xml tag별로 내용을 분류함)
    return a.title, preprocessing(a.text)      # a에 해당하는 title과 본문(불용어 제거) 출력

def preprocessing(text):
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거
    return text_article

for article in list_articles:   # list에 있는 모든 기사를 하나씩 방문
    _, text = crawl_article(article['link'])  # 그 기사의 link를 crawl_article 함수에 넣어 본문 추출
    article['text'] = text      # 추출된 본문을 list_articles에 'text'라는 속성을 새로 만들어 저장

[Crawl Article] https://www.etnews.com/20231004000053
[Crawl Article] https://www.etnews.com/20231003000006
[Crawl Article] https://www.etnews.com/20230927000112
[Crawl Article] https://www.etnews.com/20231004000004
[Crawl Article] https://www.etnews.com/20231004000003
[Crawl Article] https://www.etnews.com/20231004000002
[Crawl Article] https://www.etnews.com/20231003000103
[Crawl Article] https://www.etnews.com/20230926000252
[Crawl Article] https://www.etnews.com/20231002000054
[Crawl Article] https://www.etnews.com/20231002000053
[Crawl Article] https://www.etnews.com/20230926000326
[Crawl Article] https://www.etnews.com/20230926000306
[Crawl Article] https://www.etnews.com/20230926000298
[Crawl Article] https://www.etnews.com/20231003000057
[Crawl Article] https://www.etnews.com/20231003000019
[Crawl Article] https://www.etnews.com/20231003000017
[Crawl Article] https://www.etnews.com/20231003000087
[Crawl Article] https://www.etnews.com/20231003000085
[Crawl Article] https://www.

In [4]:
print(list_articles[0])  # 첫번째 기사를 출력(title, link, text가 모두 나오는 것을 확인)

{'title': '아이파트너즈, 혁신 스타트업 유럽 진출 지원 시동', 'link': 'https://www.etnews.com/20231004000053', 'text': "액셀러레이팅 전문기업 아이파트너즈가 국내 유망 스타트업에 유럽 진출 지원 프로그램을 제공한다.\n\n\n\n아이파트너즈(대표 김태규)와 IBK창공은 지난달부터 해외 진출을 희망하는 국내 혁신·벤처 스타트업을 대상으로 'IBK창공 유럽 액셀러레이팅' 운영을 시작했다고 4일 밝혔다.\n\n\n\n'IBK창공 유럽 액셀러레이팅'은 IBK창공과 아이파트너즈, 연구개발특구진흥재단(강소특구)이 독일 잘란트주, 한국과학기술연구원(KIST) 유럽연구소와 협업해 만든 스타트업 유럽진출 지원 프로그램이다.\n\n\n\n국내에서 선발한 유망 혁신 스타트업을 독일 잘란트주와 KIST 유럽연구소가 운영하는 스타트업 육성 인프라 'EU허브'에 보내 현지거점 마련을 비롯해 단계적으로 유럽 진출에 필요한 다양한 요소를 발굴, 지원한다.\n\n\n\n아이파트너즈와 IBK창공은 인공지능(AI), 빅데이터 등 정보통신기술(ICT)과 첨단제조, 바이오 등 혁신 분야 스타트업 11개사를 선발해 지난달 18~22일까지 5일 동안 1차 유럽 액셀러레이팅을 진행했다.\n\n\n\n임팩티브에이아이, 맵시, 원더무브 등 11개 스타트업 대표들은 국내에서 2주 동안 아이템 집중 진단, 현지화 전략 등 사전 컨설팅을 받고, 이어 독일로 건너가 잘란트주 EU허브에서 유럽 투자자를 대상으로 투자유치 IR, 데모데이 등 유럽 액셀러레이팅 프로그램에 참가했다.\n\n\n\n이 가운데 두개 스타트업은 유럽 연락사무소 및 법인 설립을 검토하기로 했고, 스타트업 한 곳은 잘란트주 대학과 자사제품 공급계약을 체결했다.\n\n\n\n김태규 대표는 “이번 1차 프로그램 운영 성과를 토대로 내년 상반기에 2차 프로그램을 계획하고 있다”며 “국내 창업지원 육성 전문기관, 기존 유럽 진출 기업 및 현지 기관과 긴밀한 네트워크를 구축해 국내 유망 스타트업의 글

In [ ]:
# feedparser:
# 주로 피드로부터 제목, 링크, 요약과 같은 메타데이터를 추출합니다.
# 주로 뉴스 헤드라인이나 블로그 게시물의 링크를 가져올 때 사용됩니다.
# Article.parser:
# 웹 페이지의 실제 본문 내용을 추출합니다.
# 주로 기사 내용, 블로그 게시물 내용, 제품 설명 등 웹 페이지의 텍스트 콘텐츠를 가져올 때 사용됩니다.

In [5]:
## [3] 모든 본문 text에서 명사 (키워드, 빈도수) 추출
def get_keywords(text, nKeywords=10):   # 키워드 추출 함수, 디폴트로 10개 지정, 키워드로 10개 가져
    spliter = Okt()              # konlpy에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
    nouns = spliter.nouns(text)  # spliter에 의해서 nouns함수를 불러 text를 넣으면 그 text의 명사만 추출 / 여러개의 명사를 뽑을 땐 nouns, 하나만 뽑을 땐 noun
    count = Counter(nouns)       # 추출된 명사들의 출현빈도 추출
    list_keywords = []           # 비어있는 키워드 리스트 생성

    for n, c in count.most_common(nKeywords):  # 가장 출현빈도가 높은 명사부터 순차적으로 nKeywords 횟수만큼 출력
        item = {'keyword':n, 'count':c}        # 리스트에 저장은 {'keyword', 'count'} 형식으로 저장
        list_keywords.append(item)             # 이런 저장된 포켓으로 이를 list_keywords에 붙여 저장

    return list_keywords

for article in list_articles:    # 모든 기사를 돌아다니며 text에서 명사 추출, 키워드/빈도 추출
    keywords = get_keywords(article['text'])  # get_keywords 함수로 키워드/빈도 추출
    article['keywords'] = keywords             # 추출된 키워드/빈도를 list_articles의 'keywords'로 저장

print(list_articles[0])          # 첫번째 기사 출력(title, link, text, keywords(키워드/빈도 최대 nKeywords 횟수))

{'title': '아이파트너즈, 혁신 스타트업 유럽 진출 지원 시동', 'link': 'https://www.etnews.com/20231004000053', 'text': "액셀러레이팅 전문기업 아이파트너즈가 국내 유망 스타트업에 유럽 진출 지원 프로그램을 제공한다.\n\n\n\n아이파트너즈(대표 김태규)와 IBK창공은 지난달부터 해외 진출을 희망하는 국내 혁신·벤처 스타트업을 대상으로 'IBK창공 유럽 액셀러레이팅' 운영을 시작했다고 4일 밝혔다.\n\n\n\n'IBK창공 유럽 액셀러레이팅'은 IBK창공과 아이파트너즈, 연구개발특구진흥재단(강소특구)이 독일 잘란트주, 한국과학기술연구원(KIST) 유럽연구소와 협업해 만든 스타트업 유럽진출 지원 프로그램이다.\n\n\n\n국내에서 선발한 유망 혁신 스타트업을 독일 잘란트주와 KIST 유럽연구소가 운영하는 스타트업 육성 인프라 'EU허브'에 보내 현지거점 마련을 비롯해 단계적으로 유럽 진출에 필요한 다양한 요소를 발굴, 지원한다.\n\n\n\n아이파트너즈와 IBK창공은 인공지능(AI), 빅데이터 등 정보통신기술(ICT)과 첨단제조, 바이오 등 혁신 분야 스타트업 11개사를 선발해 지난달 18~22일까지 5일 동안 1차 유럽 액셀러레이팅을 진행했다.\n\n\n\n임팩티브에이아이, 맵시, 원더무브 등 11개 스타트업 대표들은 국내에서 2주 동안 아이템 집중 진단, 현지화 전략 등 사전 컨설팅을 받고, 이어 독일로 건너가 잘란트주 EU허브에서 유럽 투자자를 대상으로 투자유치 IR, 데모데이 등 유럽 액셀러레이팅 프로그램에 참가했다.\n\n\n\n이 가운데 두개 스타트업은 유럽 연락사무소 및 법인 설립을 검토하기로 했고, 스타트업 한 곳은 잘란트주 대학과 자사제품 공급계약을 체결했다.\n\n\n\n김태규 대표는 “이번 1차 프로그램 운영 성과를 토대로 내년 상반기에 2차 프로그램을 계획하고 있다”며 “국내 창업지원 육성 전문기관, 기존 유럽 진출 기업 및 현지 기관과 긴밀한 네트워크를 구축해 국내 유망 스타트업의 글

In [8]:
import math

In [10]:
## [4] 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력
query = input() # 쿼리 입력받음

# IDF 계산을 위한 함수
def calculate_idf(query, list_articles):
    doc_count = len(list_articles)
    doc_containing_query = sum(1 for article in list_articles if search_articles(query, article['keywords']) > 0)
    if doc_containing_query == 0:
        return 0
    return math.log(doc_count / doc_containing_query)

# TF-IDF 계산 함수
def calculate_tfidf(tf, idf):
    return tf * idf

# 빈도수에 따라 기사를 정렬하는 함수
def sort_articles_by_frequency(articles):
    return sorted(articles, key=lambda x: search_articles(query, x['keywords']), reverse=True)

# 기사 리스트를 빈도수에 따라 정렬
sorted_articles = sort_articles_by_frequency(list_articles)

for article in sorted_articles:
    nQuery = search_articles(query, article['keywords']) # TF 계산
    nIDF = calculate_idf(query, sorted_articles)  # IDF 계산
    nTFIDF = calculate_tfidf(nQuery, nIDF)  # TF-IDF 계산

    if nQuery != 0:
        print('[TF]', nQuery, '[IDF]', nIDF, '[TF-IDF]', nTFIDF, '[Title]', article['title'], '[url]', article['link'])



# def search_articles(query, list_keywords):   # 쿼리가 키워드 리스트에 있으면 회수를 출력(없으면 0)
#     nWords = 0    # 아래의 if문에 걸리지 않으면(즉 쿼리가 키워드에 없으면) 0을 출력하기 위함
#     for kw in list_keywords:         # 키워드 리스트 검색
#         if eq(query, kw['keyword']): # 쿼리와 동일한 키워드가 존재한다면    # eq 대신 == 사용 가능
#             nWords = kw['count']     # 그 키워드에 해당하는 count가 nWords값이 됨
#     return nWords # 결과적으로 쿼리의 출현 횟수를 출력

# # 빈도수에 따라 기사를 정렬하는 함수
# def sort_articles_by_frequency(articles):
#     return sorted(articles, key=lambda x: search_articles(query, x['keywords']), reverse=True)

# # 기사 리스트를 빈도수에 따라 정렬
# sorted_articles = sort_articles_by_frequency(list_articles)

# for article in sorted_articles:  # 모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오나 검색
#     nQuery = search_articles(query, article['keywords']) # search_articles 함수로 빈도수 추출
#     if nQuery != 0:            # 쿼리가 키워드에 존재하지 않는 경우는 출력하지 않
#         print('[TF]', nQuery, '[Title]', article['title'], '[url]', article['link'])

반도체
[TF] 17 [IDF] 3.2321210516182215 [TF-IDF] 54.946057877509766 [Title] [미래 반도체 유니콘 열전]〈4〉퀄리타스반도체, 대규모 R&D로 '글로벌 인터커넥트 IP 기업' 도전 [url] https://www.etnews.com/20231003000006
[TF] 12 [IDF] 3.2321210516182215 [TF-IDF] 38.785452619418656 [Title] 美, 반도체 투자 3억달러 미만 기업도 지원금…산업부 “韓 기업 지원” [url] https://www.etnews.com/20230930000218
[TF] 6 [IDF] 3.2321210516182215 [TF-IDF] 19.392726309709328 [Title] 산업부, '무역·기술 안보 종합전략' 세운다…연내 포럼 구성 [url] https://www.etnews.com/20231004000002
